<a href="https://colab.research.google.com/github/abhiyantaabhishek/speed_bump/blob/master/yolo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive

In [0]:
!google-drive-ocamlfuse drive

In [0]:
!pip install tensorflow

In [0]:
ls drive/bump_dataset

'Copy of Untitled0.ipynb'   ir_det_dataset/   ir_det_dataset.tar.gz


In [0]:
cd drive/bump_dataset

/content/drive/bump_dataset


In [0]:
pwd

'/content/drive/bump_dataset'

In [0]:
!tar -zxvf ir_det_dataset.tar.gz

ir_det_dataset/
ir_det_dataset/labels/
ir_det_dataset/labels/rgb/
ir_det_dataset/labels/rgb/002534.txt
ir_det_dataset/labels/rgb/000911.txt
ir_det_dataset/labels/rgb/006839.txt
ir_det_dataset/labels/rgb/006014.txt
ir_det_dataset/labels/rgb/007127.txt
ir_det_dataset/labels/rgb/002441.txt
ir_det_dataset/labels/rgb/001632.txt
ir_det_dataset/labels/rgb/003551.txt
ir_det_dataset/labels/rgb/001423.txt
ir_det_dataset/labels/rgb/000115.txt
ir_det_dataset/labels/rgb/000479.txt
ir_det_dataset/labels/rgb/006588.txt
ir_det_dataset/labels/rgb/003573.txt
ir_det_dataset/labels/rgb/000293.txt
ir_det_dataset/labels/rgb/005237.txt
ir_det_dataset/labels/rgb/007178.txt
ir_det_dataset/labels/rgb/000940.txt
ir_det_dataset/labels/rgb/002550.txt
ir_det_dataset/labels/rgb/005570.txt
ir_det_dataset/labels/rgb/000439.txt
ir_det_dataset/labels/rgb/001379.txt
ir_det_dataset/labels/rgb/000847.txt
ir_det_dataset/labels/rgb/002245.txt
ir_det_dataset/labels/rgb/002862.txt
ir_det_dataset/labels/rgb/005820.txt
ir_det_da